# Colab 데이터 로드

In [1]:
#Step 1.분석할 데이터가 저장된 파일을 불러와서 변수에 할당합니다.
from google.colab import files
myfile = files.upload()
import io
import pandas as pd
#pd.read_csv로 csv파일 불러오기
src_data = pd.read_csv(io.BytesIO(myfile['와인.csv']),
                       encoding='cp949')
src_data

Saving 와인.csv to 와인.csv


,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 로컬 데이터 로드

In [3]:
import pandas as pd
src_data = pd.read_csv('와인.csv', encoding='cp949')
src_data

,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 공통 실습 코드

In [5]:
#Step 2.주어진 데이터를 훈련용 데이터와 검증용 데이터로 나눕니다
import numpy as np
data = src_data[ ['알콜도수','당도','산도_ph']].to_numpy()
target = src_data['종류'].to_numpy()
print(data)
print(target)

from sklearn.model_selection import train_test_split
훈련용_data , 테스트용_data , 훈련용_target , 테스트용_target = train_test_split(
    data , target , test_size=0.2 , random_state=10)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]]
['레드와인' '레드와인' '레드와인' ... '화이트와인' '화이트와인' '화이트와인']


In [6]:
# 교차검증 없이 모델 검증합니다.
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=10)
dt.fit(훈련용_data , 훈련용_target)
print(dt.score(훈련용_data , 훈련용_target))
print(dt.score(테스트용_data , 테스트용_target))

0.9974985568597268
0.8515384615384616


In [4]:
# Step 3.K-Fold 교차 검증 수행(K값 자율 설정)
# cross_validate , cross_val_score 출력
from sklearn.model_selection import cross_validate , cross_val_score
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=10)
scores_1 = cross_validate(dt , 훈련용_data , 훈련용_target,cv=5)
scores_2 = cross_val_score(dt, 훈련용_data , 훈련용_target,cv=5)
print('cross_validate 결과:', np.mean(scores_1['test_score']))
print('cross_val_score 결과:', np.mean(scores_2) )

cross_validate 결과: 0.8578044717553862
cross_val_score 결과: 0.8578044717553862


In [7]:
#그리드서치를 사용하여 하이퍼 파라미터 튜닝하기 
from sklearn.model_selection import GridSearchCV
parm = {'max_depth': range(1,10,1) ,
        'min_impurity_decrease': np.arange(0.0001,0.001 , 0.0001),
        'min_samples_split' : range(2,100,10) }
gs = GridSearchCV(DecisionTreeClassifier(random_state=50) , parm , n_jobs=-1)
gs.fit(훈련용_data , 훈련용_target)
print(gs.best_params_)

{'max_depth': 9, 'min_impurity_decrease': 0.0005, 'min_samples_split': 2}


c:\Users\dusgn\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [8]:
print(gs.best_params_)
dt = gs.best_estimator_
print(dt.score(훈련용_data , 훈련용_target))
print(dt.score(테스트용_data , 테스트용_target))

{'max_depth': 9, 'min_impurity_decrease': 0.0005, 'min_samples_split': 2}
0.8916682701558591
0.8653846153846154
